<a href="https://colab.research.google.com/github/FelipiFagundes/Aula_4_imersao_Alura/blob/main/Aula5_imersao_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q -U google-generativeai

In [6]:
#configurações iniciais e importações

import numpy as np
import pandas as pd
import google.generativeai as genai

from google.colab import userdata
api_key = userdata.get('SECRET_KEY')

genai.configure(api_key=api_key)

In [9]:
for m in genai.list_models():
  if "embedContent" in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004


Exemplos de embedding

In [10]:
title = "A próxima geração de IA e Desenvolvedores"
sample_text = ("Título: A próxima geração de IA e Desenvolvedores"
"\n"
"Artigo completo:\n"
"\n"
"Gemini API e Google AI Studio: A próxima geração de IA e Desenvolvedores")

embeddings = genai.embed_content(model="models/embedding-001",
                                 content=sample_text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")

print(embeddings)

{'embedding': [0.03548467, -0.017110717, -0.010312904, -0.012438664, 0.08495483, -0.0044972026, -0.023503188, 0.0009849787, 0.059021235, 0.06529612, 0.008033685, 0.0245022, -0.041851874, -0.016166616, 0.013423983, -0.020382876, 0.018459763, -0.0028214261, -0.021982692, 0.0036663043, -0.0004358115, 0.0009833965, -0.01918085, -0.04362587, -0.027216729, 0.025897682, 0.013877325, -0.045248896, -0.03688535, 0.018298492, -0.038457524, 0.047368348, -0.026824435, 0.007202248, -0.018242974, -0.06418138, -0.0338955, -0.056741294, 0.002024549, -0.0003461698, 0.014346634, -0.086375445, 0.003760442, 0.005714629, 0.003169731, -0.032645404, 0.03433096, 0.052941788, 0.030197306, -0.06326714, 0.030602094, 0.038835224, 0.083263926, -0.03474508, -0.009084008, 0.0049799904, 0.050556947, -0.04066794, 0.01319766, 0.009850423, 0.007444044, -0.0020691932, -0.0202093, 0.051748212, -0.0071161026, -0.040975224, -0.03687071, -0.0062482245, 0.03304733, 0.038291253, 0.030567976, 0.013497569, 0.078053445, -0.0670002

In [11]:
# Listagem de documentos que serão buscados
DOCUMENT1 = {
    "Título": "Operação do sistema de controle climático",
    "Conteúdo": "O Googlecar tem um sistema de controle climático que permite ajustar a temperatura e o fluxo de ar no carro. Para operar o sistema, use os botões no painel."
}

DOCUMENT2 = {
    "Título": "Touchscreen",
    "Conteúdo": "O seu Googlecar tem uma grande tela sensível ao toque que fornece acesso a uma variedade de recursos, incluindo navegação, entretenimento e comunicação."
}

DOCUMENT3 = {
    "Título": "Mudança de marchas",
    "Conteúdo": "Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada."
}

documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3]

In [14]:
df = pd.DataFrame(documents)
df.columns = ["Título", "Conteúdo"]
df

,Título,Conteúdo
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...


In [15]:
model = "models/embedding-001"

In [16]:
def embed_fn(title,text):
  return genai.embed_content(model=model,
                                 content=text,
                                 title=title,
                                 task_type="RETRIEVAL_DOCUMENT")["embedding"]

In [17]:
df["Embeddings"] = df.apply(lambda row : embed_fn(row["Título"], row["Conteúdo"]), axis=1)
df

,Título,Conteúdo,Embeddings
0,Operação do sistema de controle climático,O Googlecar tem um sistema de controle climáti...,"[-0.019505827, -0.027598742, -0.04724778, 0.03..."
1,Touchscreen,O seu Googlecar tem uma grande tela sensível a...,"[0.019957777, -0.046034534, -0.0146649685, -0...."
2,Mudança de marchas,Seu Googlecar tem uma transmissão automática. ...,"[0.00018316552, -0.03903683, 0.0017067897, 0.0..."


In [18]:
def gerar_buscar_consulta(consulta, base, model):
  embedding_da_consulta = genai.embed_content(model=model,
                                 content=consulta,
                                 task_type="RETRIEVAL_QUERY")["embedding"]

  produtos_escalares = np.dot(np.stack(df["Embeddings"]), embedding_da_consulta)

  indice = np.argmax(produtos_escalares)
  return df.iloc[indice]["Conteúdo"]

In [21]:
consulta = "Como trocar a marcha no carro do google?"

trecho = gerar_buscar_consulta(consulta, df, model)
print(trecho)

Seu Googlecar tem uma transmissão automática. Para trocar as marchas, basta mover a alavanca de câmbio para a posição desejada.


In [26]:
generation_config = {
    "temperature": 0.8,
    "candidate_count": 1,
}

In [27]:
prompt = f"Reescreva este trecho de ma maneira descontraída, sem adicionar informações que não fazem parte do texto:{trecho}"

model_2 = genai.GenerativeModel("gemini-1.0-pro",
                                generation_config=generation_config)

response = model_2.generate_content(prompt)
print(response.text)

Yo, seu Googlecar é automático, tipo, o máximo! Tá a fim de trocar de marcha? É só dar um rolê com a alavanca pra onde você quiser!
